# My COVID-19 Overview
> Tracking coronavirus total cases, deaths and new cases by country.  Additionally, a detailed view is provided for the US by state.

- author: Srikanth Chinmay (inspired by Pratap Vardhan @ http://covid19dashboards.com) 
- categories: [covid-19]
- image: images/covid-overview.png
- permalink: /covid-overview/
- toc: true
- comments: false


# Part One: Statistics By Country

In [1]:
#hide
print('''
Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/
''')


Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/



In [0]:
#hide
import numpy as np
import pandas as pd
from jinja2 import Template
from IPython.display import HTML

In [0]:
#hide

# FETCH
import getpass
base_url = 'https://raw.githubusercontent.com/pratapvardhan/notebooks/master/covid19/'
base_url = '' if (getpass.getuser() == 'Pratap Vardhan') else base_url
paths = {
    'mapping': base_url + 'mapping_countries.csv',
    'overview': base_url + 'overview.tpl'
}

def get_mappings(url):
    df = pd.read_csv(url)
    return {
        'df': df,
        'replace.country': dict(df.dropna(subset=['Name']).set_index('Country')['Name']),
        'map.continent': dict(df.set_index('Name')['Continent'])
    }

mapping = get_mappings(paths['mapping'])

def get_template(path):
    from urllib.parse import urlparse
    if bool(urlparse(path).netloc):
        from urllib.request import urlopen
        return urlopen(path).read().decode('utf8')
    return open(path).read()

def get_frame(name):
    url = (
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
        f'csse_covid_19_time_series/time_series_19-covid-{name}.csv')
    df = pd.read_csv(url)
    # rename countries
    df['Country/Region'] = df['Country/Region'].replace(mapping['replace.country'])
    return df

def get_dates(df):
    dt_cols = df.columns[~df.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]
    LAST_DATE_I = -1
    # sometimes last column may be empty, then go backwards
    for i in range(-1, -len(dt_cols), -1):
        if not df[dt_cols[i]].fillna(0).eq(0).all():
            LAST_DATE_I = i
            break
    return LAST_DATE_I, dt_cols

In [0]:
#hide
COL_REGION = 'Country/Region'
# Confirmed, Recovered, Deaths
df = get_frame('Confirmed')
# dft_: timeseries, dfc_: today country agg
dft_cases = df
dft_deaths = get_frame('Deaths')
dft_recovered = get_frame('Recovered')
LAST_DATE_I, dt_cols = get_dates(df)

dt_today = dt_cols[LAST_DATE_I]
dt_ago = dt_cols[LAST_DATE_I-1]

dfc_cases = dft_cases.groupby(COL_REGION)[dt_today].sum()
dfc_deaths = dft_deaths.groupby(COL_REGION)[dt_today].sum()
dfc_recovered = dft_recovered.groupby(COL_REGION)[dt_today].sum()
dfp_cases = dft_cases.groupby(COL_REGION)[dt_ago].sum()
dfp_deaths = dft_deaths.groupby(COL_REGION)[dt_ago].sum()
dfp_recovered = dft_recovered.groupby(COL_REGION)[dt_ago].sum()

In [5]:
#hide
df_table = (pd.DataFrame(dict(
    Cases=dfc_cases, Deaths=dfc_deaths, Recovered=dfc_recovered,
    PCases=dfp_cases, PDeaths=dfp_deaths, PRecovered=dfp_recovered))
             .sort_values(by=['Cases', 'Deaths'], ascending=[False, False])
             .reset_index())
for c in 'Cases, Deaths, Recovered'.split(', '):
    df_table[f'{c} (+)'] = (df_table[c] - df_table[f'P{c}']).clip(0)  # DATA BUG
df_table['Fatality Rate'] = (100 * df_table['Deaths'] / df_table['Cases']).round(1)
df_table['Continent'] = df_table['Country/Region'].map(mapping['map.continent'])
df_table.head(15)

,Country/Region,Cases,Deaths,Recovered,PCases,PDeaths,PRecovered,Cases (+),Deaths (+),Recovered (+),Fatality Rate,Continent
0,China,81397,3265,72362,81305,3259,71857,92,6,505,4.0,Asia
1,Italy,59138,5476,7024,53578,4825,6072,5560,651,952,9.3,Europe
2,US,33272,417,0,25489,307,0,7783,110,0,1.3,North America
3,Spain,28768,1772,2575,25374,1375,2125,3394,397,450,6.2,Europe
4,Germany,24873,94,266,22213,84,233,2660,10,33,0.4,Europe
5,Iran,21638,1685,7931,20610,1556,7635,1028,129,296,7.8,Asia
6,France,16176,674,2206,14431,562,12,1745,112,2194,4.2,Europe
7,South Korea,8897,104,2909,8799,102,1540,98,2,1369,1.2,Asia
8,Switzerland,7245,98,131,6575,75,15,670,23,116,1.4,Europe
9,United Kingdom,5741,282,67,5067,234,67,674,48,0,4.9,Europe


In [6]:
#hide
# world, china, europe, us
metrics = ['Cases', 'Deaths', 'Recovered', 'Cases (+)', 'Deaths (+)', 'Recovered (+)']
s_china = df_table[df_table['Country/Region'].eq('China')][metrics].sum().add_prefix('China ')
s_us = df_table[df_table['Country/Region'].eq('US')][metrics].sum().add_prefix('US ')
s_eu = df_table[df_table['Continent'].eq('Europe')][metrics].sum().add_prefix('EU ')
summary = {'updated': pd.to_datetime(dt_today), 'since': pd.to_datetime(dt_ago)}
summary = {**summary, **df_table[metrics].sum(), **s_china, **s_us, **s_eu}
summary

{'Cases': 335955,
 'Cases (+)': 31431,
 'China Cases': 81397,
 'China Cases (+)': 92,
 'China Deaths': 3265,
 'China Deaths (+)': 6,
 'China Recovered': 72362,
 'China Recovered (+)': 505,
 'Deaths': 14632,
 'Deaths (+)': 1660,
 'EU Cases': 169296,
 'EU Cases (+)': 18493,
 'EU Deaths': 8807,
 'EU Deaths (+)': 1318,
 'EU Recovered': 12795,
 'EU Recovered (+)': 3799,
 'Recovered': 97704,
 'Recovered (+)': 6214,
 'US Cases': 33272,
 'US Cases (+)': 7783,
 'US Deaths': 417,
 'US Deaths (+)': 110,
 'US Recovered': 0,
 'US Recovered (+)': 0,
 'since': Timestamp('2020-03-21 00:00:00'),
 'updated': Timestamp('2020-03-22 00:00:00')}

In [7]:
#hide
dft_ct_cases = dft_cases.groupby(COL_REGION)[dt_cols].sum()
dft_ct_new_cases = dft_ct_cases.diff(axis=1).fillna(0).astype(int)
dft_ct_new_cases.head()

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20
Country/Region,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,3,0,1,2,0,0,4,5,5,1,0,0,2,0,16
Albania,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,8,2,11,10,5,4,9,4,4,5,6,6,13
Algeria,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2,2,7,0,5,0,2,1,0,0,4,2,11,11,6,6,14,13,3,49,62
Andorra,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,37,0,14,22,13,25
Angola,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0


In [8]:
#hide_input
template = Template(get_template(paths['overview']))
html = template.render(
    D=summary, table=df_table.head(20),  # REMOVE .head(20) to see all values
    newcases=dft_ct_new_cases.loc[:, dt_cols[LAST_DATE_I-50]:dt_cols[LAST_DATE_I]],
    np=np, pd=pd, enumerate=enumerate)
HTML(f'<div>{html}</div>')

# Part Two: United States, by State

In [0]:
#hide

# FETCH
import getpass
base_url = 'https://raw.githubusercontent.com/pratapvardhan/notebooks/master/covid19/'
base_url = '' if (getpass.getuser() == 'Pratap Vardhan') else base_url
paths = {
    'mapping': base_url + 'mapping_countries.csv',
    'overview': base_url + 'overview.tpl'
}

def get_mappings(url):
    df = pd.read_csv(url)
    return {
        'df': df,
        'replace.country': dict(df.dropna(subset=['Name']).set_index('Country')['Name']),
        'map.continent': dict(df.set_index('Name')['Continent'])
    }

mapping = get_mappings(paths['mapping'])

def get_template(path):
    from urllib.parse import urlparse
    if bool(urlparse(path).netloc):
        from urllib.request import urlopen
        return urlopen(path).read().decode('utf8')
    return open(path).read()

def get_frame(name):
    url = (
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
        f'csse_covid_19_time_series/time_series_19-covid-{name}.csv')
    df = pd.read_csv(url)
    # rename countries
    df['Country/Region'] = df['Country/Region'].replace(mapping['replace.country'])
    return df

def get_dates(df):
    dt_cols = df.columns[~df.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]
    LAST_DATE_I = -1
    # sometimes last column may be empty, then go backwards
    for i in range(-1, -len(dt_cols), -1):
        if not df[dt_cols[i]].fillna(0).eq(0).all():
            LAST_DATE_I = i
            break
    return LAST_DATE_I, dt_cols

In [0]:
#hide
COL_REGION = 'Province/State'   
# Confirmed, Recovered, Deaths
US_POI = [
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
    'Colorado', 'Connecticut', 'Delaware', 'Diamond Princess',
    'District of Columbia', 'Florida', 'Georgia', 'Grand Princess',
    'Guam', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
    'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
    'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
    'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
    'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma',
    'Oregon', 'Pennsylvania', 'Puerto Rico', 'Rhode Island',
    'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah',
    'Vermont', 'Virgin Islands', 'Virginia', 'Washington',
    'West Virginia', 'Wisconsin', 'Wyoming']

filter_us = lambda d: d[d['Country/Region'].eq('US') & d['Province/State'].isin(US_POI)]

df = get_frame('Confirmed')
# dft_: timeseries, dfc_: today country agg
dft_cases = df.pipe(filter_us)
dft_deaths = get_frame('Deaths').pipe(filter_us)
dft_recovered = get_frame('Recovered').pipe(filter_us)

LAST_DATE_I, dt_cols = get_dates(df)

dt_today = dt_cols[LAST_DATE_I]
dt_ago = dt_cols[LAST_DATE_I-1]

dfc_cases = dft_cases.groupby(COL_REGION)[dt_today].sum()
dfc_deaths = dft_deaths.groupby(COL_REGION)[dt_today].sum()
dfc_recovered = dft_recovered.groupby(COL_REGION)[dt_today].sum()
dfp_cases = dft_cases.groupby(COL_REGION)[dt_ago].sum()
dfp_deaths = dft_deaths.groupby(COL_REGION)[dt_ago].sum()
dfp_recovered = dft_recovered.groupby(COL_REGION)[dt_ago].sum()

In [11]:
#hide
df_table = (pd.DataFrame(dict(
    Cases=dfc_cases, Deaths=dfc_deaths, Recovered=dfc_recovered,
    PCases=dfp_cases, PDeaths=dfp_deaths, PRecovered=dfp_recovered))
             .sort_values(by=['Cases', 'Deaths'], ascending=[False, False])
             .reset_index())
for c in 'Cases, Deaths, Recovered'.split(', '):
    df_table[f'{c} (+)'] = (df_table[c] - df_table[f'P{c}']).clip(0)  # DATA BUG
df_table['Fatality Rate'] = (100 * df_table['Deaths'] / df_table['Cases']).round(1)
df_table.head(15)

,Province/State,Cases,Deaths,Recovered,PCases,PDeaths,PRecovered,Cases (+),Deaths (+),Recovered (+),Fatality Rate
0,New York,15793,117,0,11710,60,0,4083,57,0,0.7
1,Washington,1996,95,0,1793,94,0,203,1,0,4.8
2,New Jersey,1914,20,0,1327,16,0,587,4,0,1.0
3,California,1642,30,0,1364,24,0,278,6,0,1.8
4,Illinois,1049,9,0,753,6,0,296,3,0,0.9
5,Michigan,1037,9,0,788,5,0,249,4,0,0.9
6,Louisiana,837,20,0,585,16,0,252,4,0,2.4
7,Florida,830,13,0,659,13,0,171,0,0,1.6
8,Massachusetts,646,5,0,525,1,0,121,4,0,0.8
9,Texas,627,6,0,581,5,0,46,1,0,1.0


In [12]:
#hide
# world, china, europe, us
metrics = ['Cases', 'Deaths', 'Recovered', 'Cases (+)', 'Deaths (+)', 'Recovered (+)']
s_ny = df_table[df_table[COL_REGION].eq('New York')][metrics].sum().add_prefix('NY ')
s_wa = df_table[df_table[COL_REGION].eq('Washington')][metrics].sum().add_prefix('WA ')
s_ca = df_table[df_table[COL_REGION].eq('California')][metrics].sum().add_prefix('CA ')
summary = {'updated': pd.to_datetime(dt_today), 'since': pd.to_datetime(dt_ago)}
summary = {**summary, **df_table[metrics].sum(), **s_ny, **s_wa, **s_ca}
summary

{'CA Cases': 1642,
 'CA Cases (+)': 278,
 'CA Deaths': 30,
 'CA Deaths (+)': 6,
 'CA Recovered': 0,
 'CA Recovered (+)': 0,
 'Cases': 33272,
 'Cases (+)': 7783,
 'Deaths': 417,
 'Deaths (+)': 111,
 'NY Cases': 15793,
 'NY Cases (+)': 4083,
 'NY Deaths': 117,
 'NY Deaths (+)': 57,
 'NY Recovered': 0,
 'NY Recovered (+)': 0,
 'Recovered': 0,
 'Recovered (+)': 0,
 'WA Cases': 1996,
 'WA Cases (+)': 203,
 'WA Deaths': 95,
 'WA Deaths (+)': 1,
 'WA Recovered': 0,
 'WA Recovered (+)': 0,
 'since': Timestamp('2020-03-21 00:00:00'),
 'updated': Timestamp('2020-03-22 00:00:00')}

In [13]:
#hide
dft_ct_cases = dft_cases.groupby(COL_REGION)[dt_cols].sum()
dft_ct_new_cases = dft_ct_cases.diff(axis=1).fillna(0).astype(int)
dft_ct_new_cases.head()

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20
Province/State,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Alabama,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,1,6,17,10,7,32,5,48,7
Alaska,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,3,3,3,3,6
Arizona,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,3,0,0,3,1,5,2,7,18,33,40,34
Arkansas,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,5,0,6,4,6,0,11,29,34,26,43
California,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,144,33,44,61,58,86,131,141,53,201,225,187,278


In [14]:
#hide_input
template = Template(get_template(paths['overview']))
html = template.render(
    D=summary, table=df_table,
    newcases=dft_ct_new_cases.loc[:, dt_cols[LAST_DATE_I-15]:dt_cols[LAST_DATE_I]],
    COL_REGION=COL_REGION,
    KPI_CASE='US',
    KPIS_INFO=[
        {'title': 'New York', 'prefix': 'NY'},
        {'title': 'Washington', 'prefix': 'WA'},
        {'title': 'California', 'prefix': 'CA'}],
    LEGEND_DOMAIN=[5, 50, 500, np.inf],
    np=np, pd=pd, enumerate=enumerate)
HTML(f'<div>{html}</div>')

Visualizations by [Pratap Vardhan](https://twitter.com/PratapVardhan)[^1]

[^1]: Source: ["COVID-19 Data Repository by Johns Hopkins CSSE"](https://systems.jhu.edu/research/public-health/ncov/) [GitHub repository](https://github.com/CSSEGISandData/COVID-19). Link to [notebook](https://github.com/pratapvardhan/notebooks/blob/master/covid19/covid19-overview.ipynb), [orignal interactive](https://gramener.com/enumter/covid19/)